In [1]:
from unidecode import unidecode

# 1.

In [1]:
names = [
    "Rodríguez Vela, María Camino",
    "Maria Del Camino Rodriguez Vela",
    "Claudio A. De La Riva Alvarez",
    "Riva Alvarez, Claudio De La",
]

In [37]:
def clean_names(name):
    # eliminamos acentos
    name = unidecode(name.lower())
    # dividimos el nombre si tiene coma y le damos la vuelta
    if "," in name:
        name = name.split(",")
        name.reverse()
        name = list(map(lambda name: name.strip(), name))
        name = " ".join(name)

    # eliminamos palabras para unificar formatos
    name = name.split(" ")
    palabras_a_eliminar = ["de", "del", "de la", "el", "la", "-"]
    for x in palabras_a_eliminar:
        if x in name:
            name.remove(x)

    return name

In [38]:
for name in names:
    name = clean_names(name)
    print(name)

['maria', 'camino', 'rodriguez', 'vela']
['maria', 'camino', 'rodriguez', 'vela']
['claudio', 'a.', 'riva', 'alvarez']
['claudio', 'riva', 'alvarez']


# 2.

In [9]:
import json

with open("../../thesis.json", "r") as f:
    data = json.load(f)

In [10]:
# obtengo un conjunto con los nombres únicos de directores de tesis
directors_names_set = set()

for year in data.keys():
    for id in data[year].keys():
        names = data[year][id]["directors"]
        directors_names_set.update(names)

# guardo los nombres en listas para poder iterarlo
directors_names_list = list(directors_names_set)

In [14]:
def unify_duplicated_names(name1, name2):
    # quitamos acentos y pasamos a minusculas
    name1 = unidecode(name1.lower())
    name2 = unidecode(name2.lower())
    # splitteamos
    name1 = name1.split(" ")
    name2 = name2.split(" ")

    counter = 0
    for word in name1:
        # nos aseguramos de que el numero de veces que aparece la palabra
        # es el mismo en los dos nombres. Esto es importante en personas
        # cuyos apellidos son el mismo dos veces
        # P. ej: pablo gonzalez gonzalez
        if name1.count(word) == name2.count(word):
            counter += name2.count(word)

    if counter == min(len(name1), len(name2)):
        name1 = " ".join(name1)
        name1 = name1.title()

        name2 = " ".join(name2)
        name2 = name2.title()

        return name2

In [15]:
# itero sobre todos los nombres del json (autores y directores)
for year in data.keys():
    for id in data[year].keys():
        # procesamos autor
        author = data[year][id]["author"]
        author = unidecode(author)

        if "," in author:
            author = author.split(",")
            author.reverse()
            author = list(map(lambda name: name.strip(), author))
            author = " ".join(author)

        data[year][id]["author"] = author

        for director in directors_names_list:
            result_name = unify_duplicated_names(author, director)
            if result_name:
                data[year][id]["author"] = result_name

In [16]:
data

{'2024': {'2024-1': {'title': 'COLMENA: Modelo para la asistencia en la programación potenciado por la tecnología basado en análisis de datos',
   'author': 'Carlos Fernandez Medina',
   'directors': ['Juan Ramon Perez Perez']}},
 '2023': {'2023-1': {'title': 'Análisis de imagen multiespectral y segmentación semántica para la monitorización medioambiental',
   'author': 'Óscar Díaz Pedrayes',
   'directors': ['Ruben Usamentiaga Fernandez',
    'Daniel Fernando Garcia Martinez']},
  '2023-2': {'title': 'Detección de objetos utilizando modelos de aprendizaje profundo sobre dispositivos de edge computing',
   'author': 'Darío González Lema',
   'directors': ['Daniel Fernando Garcia Martinez',
    'Ruben Usamentiaga Fernandez']},
  '2023-3': {'title': 'Explainable condition monitoring from imprecise information',
   'author': 'Nahuel Alejandro Costa Cortéz',
   'directors': ['Luciano Sanchez Ramos']},
  '2023-4': {'title': 'Maintenance of the logical consistency in Cassandra',
   'author':